In [5]:
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.stats import yeojohnson, shapiro

In [6]:
gdf = gpd.read_file(r"../../Data/Processed/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_final.shp")
print(gdf)

      fid              fname  year    id type                sdate  \
0      29  Abrantes_09082017  2017   331    z  2017-08-09 18:00:00   
1      29  Abrantes_09082017  2017   334    p  2017-08-09 18:00:00   
2      29  Abrantes_09082017  2017   335    p  2017-08-09 23:00:00   
3      29  Abrantes_09082017  2017   336    p  2017-08-09 23:00:00   
4      29  Abrantes_09082017  2017   333    z  2017-08-10 12:00:00   
...   ...                ...   ...   ...  ...                  ...   
3350  135   Vinhais_01092024  2024  2795    p                  NaT   
3351  135   Vinhais_01092024  2024  2796    p                  NaT   
3352  135   Vinhais_01092024  2024  2797    p                  NaT   
3353  135   Vinhais_01092024  2024  2798    p                  NaT   
3354  135   Vinhais_01092024  2024  2799    p                  NaT   

                    edate      inidoy      enddoy    source  ...    ros_p_lg4  \
0                     NaT   -1.000000   -1.000000      fops  ...          NaN 

In [ ]:
gdf = gdf[~gdf['fire_rank'].isin([-3, -2, -1, 3])]
gdf = gdf.drop(columns=['fire_rank'])
print(gdf)

      fid              fname  year    id type                sdate  \
1      29  Abrantes_09082017  2017   334    p  2017-08-09 18:00:00   
2      29  Abrantes_09082017  2017   335    p  2017-08-09 23:00:00   
5      29  Abrantes_09082017  2017   338    p  2017-08-10 12:00:00   
8      29  Abrantes_09082017  2017   340    p  2017-08-10 14:30:00   
23    113  Abrantes_29082023  2023  2669    p  2023-08-29 17:20:00   
...   ...                ...   ...   ...  ...                  ...   
3343  135   Vinhais_01092024  2024  2789    p  2024-09-01 18:50:00   
3344  135   Vinhais_01092024  2024  2790    p  2024-09-01 18:50:00   
3346  135   Vinhais_01092024  2024  2792    p  2024-09-02 01:00:00   
3348  135   Vinhais_01092024  2024  2793    p  2024-09-02 15:00:00   
3349  135   Vinhais_01092024  2024  2794    p  2024-09-02 16:50:00   

                    edate      inidoy      enddoy    source  ...    ros_p_lg3  \
1     2017-08-09 23:00:00  221.750000  221.958333      fops  ...          NaN 

In [8]:
# Listas de variáveis por valor nulo especial
vars_com_no_value_1 = ['Cin_av']

vars_com_no_value_minus1 = [
    'CBH_m_av', 'LFC_hPa_av', 'CCL_hPa_av', 'EL_m_av',
    'Recirc', 'CircVar'
    ]

vars_com_no_value_minus999 = [
    'BLH_m_rt', 'gT_s_9_av', 'gT_9_8_av', 'gT_8_7_av', 'gT_7_5_av', 'gT_5_3_av',
    'wSv_9_av', 'wSdir_9_av', 'wSv_7_av', 'wSdir_7_av',
    'wSv_5_av', 'wSdir_5_av', 'wSv_1_av', 'wSdir_1_av'
]

# Substituir valores especiais por NaN conforme o tipo
gdf[vars_com_no_value_1] = gdf[vars_com_no_value_1].replace(1, float("nan"))

gdf[vars_com_no_value_minus1] = gdf[vars_com_no_value_minus1].replace(-1, float("nan"))

gdf[vars_com_no_value_minus999] = gdf[vars_com_no_value_minus999].replace(-999, float("nan"))


In [9]:
# 2. Variáveis a excluir
vars_excluir = [
    'fid', 'fname', 'year', 'id', 'type', 'sdate', 'edate',
    'source', 'zp_link', 'burn_perio', 'area', 'growth_rat', 'ros_i',
    'spdir_i', 'spdir_p', 'int_i', 'int_p', 'duration_i', 'qc', 'land_use_d'
]

# 3. Remover colunas indesejadas (se existirem)
gdf = gdf.drop(columns=[c for c in vars_excluir if c in gdf.columns])

for i, coluna in enumerate(gdf.columns, 1):
    print(f"{i:3d}. {coluna}")

# 4. remover -1s
gdf.replace(-1, np.nan, inplace=True)

  1. inidoy
  2. enddoy
  3. ros_p
  4. duration_p
  5. 1_3y_fir_p
  6. 3_8y_fir_p
  7. 8_ny_fir_p
  8. elev_av
  9. aspect_av
 10. landform
 11. fuel_model
 12. f_load_av
 13. land_use
 14. CBH_m_av
 15. HigCC_p_av
 16. LowCC_p_av
 17. MidCC_p_av
 18. TotCC_p_av
 19. BLH_m_av
 20. Cape_av
 21. Cin_av
 22. sW_7_av
 23. sW_28_av
 24. sW_100_av
 25. sW_289_av
 26. gp_m2s2_av
 27. t_2m_C_av
 28. d_2m_C_av
 29. sP_hPa_av
 30. wv10_kh_av
 31. wdir10_av
 32. wv100_k_av
 33. wdir100_av
 34. rh_2m_av
 35. VPD_Pa_av
 36. dfmc_av
 37. sW_1m_av
 38. sW_3m_av
 39. LCL_hPa_av
 40. LCL_m_av
 41. HDW_av
 42. Haines_av
 43. wSv_9_av
 44. wSdir_9_av
 45. wSv_7_av
 46. wSdir_7_av
 47. wSv_5_av
 48. wSdir_5_av
 49. wSv_1_av
 50. wSdir_1_av
 51. gT_s_9_av
 52. gT_9_8_av
 53. gT_8_7_av
 54. gT_7_5_av
 55. gT_5_3_av
 56. CMLG_av
 57. LFC_hPa_av
 58. CCL_hPa_av
 59. EL_m_av
 60. VentIdx_av
 61. LiftIdx_av
 62. gp_950_av
 63. gp_850_av
 64. gp_700_av
 65. gp_500_av
 66. gp_300_av
 67. rh_950_av
 68. rh_850_av

In [10]:
gdf.to_file(r"../../Data/Processed/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_model.shp")